# ETL Accidentes de Tránsito Reportados en Antioquia

In [1]:
# %pip install pandas
# %pip install unidecode

In [2]:
#  Importar librerias
import pandas as pd
from unidecode import unidecode
import re

In [3]:
# Cargar datos
df = pd.read_csv("Accidentes_de_transito_reportados_en_Antioquia.csv")

In [4]:
# TRANSFORMACION Y LIMPIEZA DE DATOS
# FECHA ACCIDENTE
# CONVERTIR LA FECHA A FORMATO DATETIME
df["FECHA ACCIDENTE"] = pd.to_datetime(df["FECHA ACCIDENTE"], format="%d/%m/%Y")

# OBTENER AÑO, MES Y DIA
df["AÑO"] = df["FECHA ACCIDENTE"].dt.year
df["MES"] = df["FECHA ACCIDENTE"].dt.month
df["DIA"] = df["FECHA ACCIDENTE"].dt.day

# OBTENER DIA DE LA SEMANA
df["DIA DE LA SEMANA"] = df["FECHA ACCIDENTE"].dt.day_name()

# TRADUCIR EL DIA DE LA SEMANA A ESPAÑOL Y PASARLOS A MAYUSCULAS
dicc_espanol = {
    "Monday": "LUNES",
    "Tuesday": "MARTES",
    "Wednesday": "MIERCOLES",
    "Thursday": "JUEVES",
    "Friday": "VIERNES",
    "Saturday": "SABADO",
    "Sunday": "DOMINGO",
}

# MAPEAR LA COLUMNA DEL DIA DE LA SEMANA CON EL DICCIONARIO
df["DIA DE LA SEMANA"] = df["DIA DE LA SEMANA"].map(dicc_espanol)

In [5]:
# HORA ACCIDENTE
# CONVERTIR LA HORA A FORMATO TIME

df["HORA ACCIDENTE"] = pd.to_datetime(
    df["HORA ACCIDENTE"], format="%H:%M", errors="coerce"
).dt.time

In [6]:
# GRAVEDAD
# MAPEAR LOS VALORES DE LA GRAVEDAD

dicc_gravedad = {"d": "SOLO DAÑOS", "h": "HERIDOS", "m": "MUERTOS"}



df["GRAVEDAD"] = df["GRAVEDAD"].map(dicc_gravedad)

In [7]:
# CLASE ACCIDENTE
# ELIMINAR TILDES Y/O CARACTERES ESPECIALES
df["CLASE ACCIDENTE"] = df["CLASE ACCIDENTE"].apply(unidecode)

In [8]:
# CAUSANTE ACCIDENTE
# PASAR A MAYUSCULAS LOS VALORES
df["CAUSANTE ACCIDENTE"] = df["CAUSANTE ACCIDENTE"].str.upper()

In [9]:
# DESCRIPCIÓN COLISIONANTE
# PASAR A MAYSUCULAS LOS VALORES Y ELIMINAR TILDES Y/O CARACTERES ESPECIALES
df["DESCRIPCIÓN COLISIONANTE"] = (
    df["DESCRIPCIÓN COLISIONANTE"].str.upper().apply(unidecode)
)

In [10]:
# DESCRIPCIÓN OBJETO FIJO
# ELIMINAR TILDES Y/O CARACTERES ESPECIALES Y LOS VALORES DE NO REPORTADO YA QUE NO APLICAN PARA LA VISUALIZACIÓN DE LOS DATOS
df["DESCRIPCIÓN OBJETO FIJO"] = df["DESCRIPCIÓN OBJETO FIJO"].apply(unidecode).replace(
    "NO REPORTADO", pd.NA
)

In [11]:
# ZONA ACCIDENTE
# ELIMINAR TILDES Y/O CARACTERES ESPECIALES Y PASAR A MAYUSCULAS
df['ZONA ACCIDENTE'] = df['ZONA ACCIDENTE'].apply(unidecode).str.upper()

In [12]:
# DESCRIPCION DE ESTADO
# ELIMINAR TILDES Y/O CARACTERES ESPECIALES Y PASAR A MAYUSCULAS
df['DESCRIPCIÓN DE ESTADO'] = df['DESCRIPCIÓN DE ESTADO'].str.upper().apply(unidecode)


In [13]:
# ÁREA ACCIDENTE
# ELIMINAR TILDES Y/O CARACTERES ESPECIALE Y PASAR A MAYUSCULAS
df['ÁREA ACCIDENTE'] = df['ÁREA ACCIDENTE'].apply(unidecode).str.upper()

In [14]:
# SECTOR ACCIDENTE
# ELIMINAR TILDES Y/O CARACTERES ESPECIALES Y PASAR A MAYUSCULAS
df['SECTOR ACCIDENTE'] = df['SECTOR ACCIDENTE'].str.upper()

In [15]:
# DESCRIPCION LOCALIZACION
# ELIMINAR TILDES Y/O CARACTERES ESPECIALES Y PASAR A MAYUSCULAS
df['DESCRIPCIÓN LOCALIZACIÓN'] = df['DESCRIPCIÓN LOCALIZACIÓN'].apply(unidecode).str.upper()

In [16]:
# ESTADO CLIMA
# ELIMINAR TILDES Y/O CARACTERES ESPECIALES Y PASAR A MAYUSCULAS
df['ESTADO CLIMA'] = df['ESTADO CLIMA'].apply(unidecode).str.upper()


In [17]:
# MUNICIPIO
# DICCIONARIO CORREGIR LOS NOMBRES DE LOS MUNICIPIOS
corrected_names = {
    "SANTA BARBARA - ANT": "SANTA BARBARA",
    "SANTABÁRBARA": "SANTA BARBARA",
    'STAFE DE ANTIOQUIA': 'SANTA FE DE ANTIOQUIA'
}

df["MUNICIPIO"] = df["MUNICIPIO"].replace(corrected_names).apply(unidecode)

In [18]:
# DIRECCION LUGAR
# ELIMIAR TILDES Y/O CARACTERES ESPECIALES
df['DIRECCIÓN LUGAR'] = df['DIRECCIÓN LUGAR'].apply(unidecode)

In [20]:
# COLUMNAS RELACIONADAS A LAS VICTIMAS
# Verificación de valores nulos y reemplazo por 0 (si es apropiado)
columns_victims = ['NUMERO VICTIMA PEATÓN', 'NUMERO VICTIMA ACOMPAÑANTE', 'NUMERO VICTIMA PASAJERO', 'NUMERO VICTIMA CONDUCTOR', 'NUMERO VICTIMA HERIDO', 'NUMERO VICTIMA MUERTO']
df[columns_victims] = df[columns_victims].fillna(0)

# Verificación de valores negativos y corrección
for column in columns_victims:
    df[column] = df[column].apply(lambda x: max(x, 0))

# Suma total de víctimas por accidente
df['TOTAL VÍCTIMAS'] = df[columns_victims].sum(axis=1)

In [21]:
df

,FECHA ACCIDENTE,HORA ACCIDENTE,GRAVEDAD,CLASE ACCIDENTE,CAUSANTE ACCIDENTE,DESCRIPCIÓN COLISIONANTE,DESCRIPCIÓN OBJETO FIJO,ZONA ACCIDENTE,DESCRIPCIÓN DE ESTADO,ÁREA ACCIDENTE,...,NUMERO VICTIMA ACOMPAÑANTE,NUMERO VICTIMA PASAJERO,NUMERO VICTIMA CONDUCTOR,NUMERO VICTIMA HERIDO,NUMERO VICTIMA MUERTO,AÑO,MES,DIA,DIA DE LA SEMANA,TOTAL VÍCTIMAS
0,2014-06-11,12:01:00,HERIDOS,VOLCAMIENTO,CICLISTA-MOTOCICLISTA,NO REPORTADO,<NA>,NO REPORTADO,PEND. NOTIFICACION,URBANA,...,0,0,0,0,0,2014,6,11,MIERCOLES,0
1,2014-03-09,09:01:00,SOLO DAÑOS,CHOQUE,CICLISTA-MOTOCICLISTA,VEHICULO,<NA>,NO REPORTADO,FALLADO,RURAL,...,0,0,0,0,0,2014,3,9,DOMINGO,0
2,2014-04-28,21:01:00,HERIDOS,CAIDA OCUPANTE,CICLISTA-MOTOCICLISTA,NO REPORTADO,<NA>,NO REPORTADO,FALLADO,RURAL,...,0,0,0,0,0,2014,4,28,LUNES,0
3,2014-05-11,15:01:00,HERIDOS,CAIDA OCUPANTE,CICLISTA-MOTOCICLISTA,NO REPORTADO,<NA>,NO REPORTADO,FALLADO,RURAL,...,0,0,0,0,0,2014,5,11,DOMINGO,0
4,2014-05-04,19:01:00,HERIDOS,ATROPELLO,CICLISTA-MOTOCICLISTA,NO REPORTADO,<NA>,NO REPORTADO,FALLADO,RURAL,...,0,0,0,0,0,2014,5,4,DOMINGO,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
841,2022-07-17,04:07:00,HERIDOS,CHOQUE,CICLISTA-MOTOCICLISTA,VEHICULO,<NA>,NO REPORTADO,PEND. NOTIFICACION,RURAL,...,0,1,0,1,0,2022,7,17,DOMINGO,2
842,2022-06-02,12:06:00,HERIDOS,CHOQUE,CICLISTA-MOTOCICLISTA,VEHICULO,<NA>,NO REPORTADO,PEND. NOTIFICACION,RURAL,...,0,0,2,2,0,2022,6,2,JUEVES,4
843,2022-03-20,13:03:00,HERIDOS,ATROPELLO,CICLISTA-MOTOCICLISTA,NO REPORTADO,<NA>,NO REPORTADO,PEND. NOTIFICACION,RURAL,...,0,1,0,1,0,2022,3,20,DOMINGO,2
844,2022-01-21,16:01:00,HERIDOS,CHOQUE,CICLISTA-MOTOCICLISTA,VEHICULO,<NA>,NO REPORTADO,PEND. NOTIFICACION,RURAL,...,0,1,1,2,0,2022,1,21,VIERNES,4
